In [2]:
!pip install -q --upgrade pip
!pip install -U jaxlib==0.4.4+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 79.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.


In [5]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.4/539.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.7/491.7 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.6/706.6 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 20.2 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [3]:
from utils import *
from layers import two_wl_aggregation

In [11]:
import numpy
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax
import shutil
import os
from jax.example_libraries import optimizers
from jax import jit, grad, vmap
from data_preperation import data_preperation, twl_sparse_pattern, twl_sparse_edge_features


In [ ]:
# define a toy 3x3 graph
# 0---2
# | /     
# 1 


edge_list_1 = np.array([[0,0],
                      [1,1],
                      [2,2],
                      [0,2],
                      [0,1],
                      [1,2],
                      [2,0],
                      [1,0],
                      [2,1],])

A = to_dense(np.transpose(edge_list_1), 3)
print(A)

edge_features = np.array([[10.0],
                          [1.0],
                          [1.0],
                          [1.0],                        
                          [20.0],
                          [1.0],
                          [1.0],
                          [1.0],
                          [30.0]])


expected = np.array([24., 34., 44., 34., 44., 54., 44., 54., 64.], dtype="float32")

print(edge_features)

As = np.expand_dims(A, 0)
As_int = neigbourhood_intersections(As)
As_pattern = np.transpose(np.array(np.nonzero(As_int)))

edge_features = np.expand_dims(edge_features, 0)
edge_features = np.expand_dims(edge_features, 2)

print(edge_features.shape)
print(As_pattern.shape)
# (188, 28, 1, 7)
# batch x 

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
[[10.]
 [ 1.]
 [ 1.]
 [ 1.]
 [20.]
 [ 1.]
 [ 1.]
 [ 1.]
 [30.]]
(1, 9, 1, 1)
(27, 4)


In [122]:
# define a toy 3x3 graph
# 0---2
# | /     
# 1 

nb_nodes = 3

edges = np.array([[0,0],
                   [1,1],
                   [2,2],
                   [0,1],
                   [0,2],
                   [1,2]])

node_features = np.array([[10],
                         [20],
                         [30],])

pattern, edge_list = twl_sparse_pattern(edges, nb_nodes)
edge_features = twl_sparse_edge_features(node_features, edge_list.shape[0], nb_nodes)

# edge_features = np.transpose(np.array([edge_features[:,1], edge_features[:,1]]))


# edge_features = np.expand_dims(edge_features, 1)
edge_features = np.expand_dims(edge_features, 1)
edge_features = np.expand_dims(edge_features, 0)


print(pattern.shape)
print(edge_features.shape)

edge_features

(18, 3)
(1, 6, 1, 2)


Array([[[[10.,  1.]],

        [[20.,  1.]],

        [[30.,  1.]],

        [[ 0.,  1.]],

        [[ 0.,  1.]],

        [[ 0.,  1.]]]], dtype=float32)

In [243]:
# for input "kernel_fn(x1, x2, 'ntk', pattern = (p1,p2))"
# the shape of the kernel after the convolution is:
# x1 = (b1, h1, w1, c1)
# x2 = (b2, h2, w2, c2)
# k = (b1, b2, h1, h2, w1, w2) 


L_branche = stax.serial(
    stax.Conv(1, (1,1), parameterization="standard"),
    )

Gamma_branche = stax.serial(
    stax.Conv(1, (1,1), parameterization="standard"),
    two_wl_aggregation(n_nodes=3),
)

two_wl_aggregation_layer = stax.serial(
    stax.FanOut(2), 
    stax.parallel(L_branche, Gamma_branche),
    stax.FanInSum(),
    stax.Relu(),
    )

init_fn, apply_fn, kernel_fn = stax.serial(
    two_wl_aggregation_layer, # k = (b1, b2, h1, h2, w1, w2)

)


In [233]:
# initialize the paramters and the optimzer 
learning_rate =  0.0001

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, edge_features.shape)

opt_init, opt_apply, get_params = optimizers.adam(learning_rate)
opt_apply = jit(opt_apply)
state = opt_init(params)

#Example to test if the pattern in the apply function behaves as expected!

In [238]:
# only the second ellemnt from the edge features => all 6. Each is aggregated over 2 times edeges with feature 1
params_t = [[(),
  ([(np.array([[[[0.0], [0.0]]]], dtype="float32"), None)],
   [(np.array([[[[0.0], [1.0]]]], dtype="float32"), None), ()]),
  (),
  ()]]

np.squeeze(apply_fn(params_t, edge_features, pattern=pattern)) 

Array([6., 6., 6., 6., 6., 6.], dtype=float32)

In [239]:
# only the first elemnt from the edge features 
# double values for the self edges (travers self edge 2 times) 
# 30: edge 1-2 can go 1-1 (10) + 1-2 (0) + 1-2 (0) + 2-2 (20)
# 40: edge 1-3 can go 1-1 (10) + 1-3 (0) + 1-3 (0) + 3-3 (30)
# 50: edge 2-3 can go 2-2 (20) + 2-3 (0) + 2-3 (0) + 3-3 (30)
params_t = [[(),
  ([(np.array([[[[0.0], [0.0]]]], dtype="float32"), None)],
   [(np.array([[[[1.0], [0.0]]]], dtype="float32"), None), ()]),
  (),
  ()]]

np.squeeze(apply_fn(params_t, edge_features, pattern=pattern)) 

Array([20., 40., 60., 30., 40., 50.], dtype=float32)

In [246]:
kernel_fn(edge_features, edge_features, "ntk", pattern=(pattern,pattern)).shape

/usr/local/lib/python3.9/dist-packages/neural_tangents/_src/stax/branching.py:399: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


(1, 1, 1, 1, 6, 6)

# TODO: Example to test if the pattern in the kernel function behaves as expected!

In [6]:
n_nodes=2
def kernel_fn(k, *,pattern, **kwargs):
        
      num_segments = int(np.prod(np.array(k.ntk.shape)))
      
      # arrange the incoming kernel matrix as a flatt array
      # ntk_linear = np.reshape(k.ntk, (-1, 1))
      # we dont need to reshape, as take works also on the multidimensional array 

      # a double sum is one sum over the karthesian product of 
      # the sets the two sums sum.
      # pattern has columns: batch, node i, node j, node a
      # the karthesian product (A x B) ist than: 
      # b_A, i_A, j_A, a_A, b_B, i_B, j_B, a_B
      #   0,   1,   2,   3,   4,   5,   6,   7
      # rearrange the columns to: 
      # b_A, b_B, i_A, i_B, j_A, j_B, a_A, a_B
      #   0,   1,   2,   3,   4,   5,   6,   7
      patterns = row_wise_karthesian_prod(pattern[0], pattern[1])       
      patterns = patterns[:,[0,4,1,5,2,6,3,7]]
      # "Interpretation" of the kartesian product:
      # A row from the pattern array [A,B,C,D] tells us,
      # that node D is a neighboor of nodes B and C in batch A.
      # Lets consider only the rows, where columns A,B,C are equal (node e_ij)
      # column D is then the "set" of nodes the "neigborhood" aggregation
      # sum for e_ij sums.
      # Also, consider the pathern from the other graph with 
      # columns [Ab,Bb,Cb,Db]. When take only the rows where 
      # columns Ab, Bb, Cb are equal. Column Db makes then the 
      # indexes to sum over for the "neigborhood" aggregation.
      # To create the double sum we need the 
      # karthesian product of the indexes in column D and 
      # column Db.

      # note: ib <=> i bar
      e_i_j_ib_jb = linear_index(patterns[:,[0,1,2,4,3,5]], (k.ntk.shape[0], k.ntk.shape[1], n_nodes, n_nodes, n_nodes, n_nodes))
      e_i_a_ib_ab = linear_index(patterns[:,[0,1,2,6,3,7]], (k.ntk.shape[0], k.ntk.shape[1], n_nodes, n_nodes, n_nodes, n_nodes))
      e_i_a_ab_jb = linear_index(patterns[:,[0,1,2,6,7,5]], (k.ntk.shape[0], k.ntk.shape[1], n_nodes, n_nodes, n_nodes, n_nodes))
      e_a_j_ib_ab = linear_index(patterns[:,[0,1,6,4,3,7]], (k.ntk.shape[0], k.ntk.shape[1], n_nodes, n_nodes, n_nodes, n_nodes))
      e_a_j_ab_jb = linear_index(patterns[:,[0,1,6,4,7,5]], (k.ntk.shape[0], k.ntk.shape[1], n_nodes, n_nodes, n_nodes, n_nodes))

      theta_i_a_ib_ab = jax.ops.segment_sum(np.take(k.ntk, e_i_a_ib_ab), e_i_j_ib_jb, num_segments)
      theta_i_a_ab_jb = jax.ops.segment_sum(np.take(k.ntk, e_i_a_ab_jb), e_i_j_ib_jb, num_segments)
      theta_a_j_ib_ab = jax.ops.segment_sum(np.take(k.ntk, e_a_j_ib_ab), e_i_j_ib_jb, num_segments)
      theta_a_j_ab_jb = jax.ops.segment_sum(np.take(k.ntk, e_a_j_ab_jb), e_i_j_ib_jb, num_segments)

      thetas_linear = np.zeros((theta_i_a_ab_jb.shape[0],1))
      thetas_linear = np.append(thetas_linear, np.expand_dims(theta_i_a_ib_ab, 1), 1)
      thetas_linear = np.append(thetas_linear, np.expand_dims(theta_i_a_ab_jb, 1), 1)
      thetas_linear = np.append(thetas_linear, np.expand_dims(theta_a_j_ib_ab, 1), 1)
      thetas_linear = np.append(thetas_linear, np.expand_dims(theta_a_j_ab_jb, 1), 1)

      theta_linear = np.sum(thetas_linear, 1)
      theta = np.reshape(theta_linear, k.ntk.shape)
      
      return k.replace(ntk=theta,
                    is_gaussian=True,
                    is_input=False)

In [7]:
def kernel_fn_new(k, *,pattern, **kwargs):

        num_segments = int(np.prod(np.array(k.ntk.shape)))

        # arrange the incoming kernel matrix as a flatt array
        # ntk_linear = np.reshape(k.ntk, (-1, 1))
        # we dont need to reshape, as take works also on the multidimensional array

        # a double sum is one sum over the karthesian product of
        # the sets the two sums sum.
        # pattern has columns: batch, node i, node j, node a
        # the karthesian product (A x B) ist than:
        # b_A, i_A, j_A, a_A, b_B, i_B, j_B, a_B
        #   0,   1,   2,   3,   4,   5,   6,   7
        # rearrange the columns to:
        # b_A, b_B, i_A, i_B, j_A, j_B, a_A, a_B
        #   0,   1,   2,   3,   4,   5,   6,   7
        patterns = row_wise_karthesian_prod(pattern[0], pattern[1])

        # the colmns of patterns are the indices for edges i_j i_a a_j ib_jb ib_ab ab_jb
        #                                                    0   1   2     3     4     5

        nb_edges_graph_a = np.unique(patterns[:, 0]).shape[0]
        nb_edges_graph_b = np.unique(patterns[:, 3]).shape[0]

        print(nb_edges_graph_a)
        print(nb_edges_graph_b)

        e_i_j_ib_jb = np.ravel_multi_index(
            [patterns[:, 0], patterns[:, 3]], (nb_edges_graph_a, nb_edges_graph_b)
        )
        e_i_a_ib_ab = np.ravel_multi_index(
            [patterns[:, 1], patterns[:, 4]], (nb_edges_graph_a, nb_edges_graph_b)
        )
        e_i_a_ab_jb = np.ravel_multi_index(
            [patterns[:, 1], patterns[:, 5]], (nb_edges_graph_a, nb_edges_graph_b)
        )
        e_a_j_ib_ab = np.ravel_multi_index(
            [patterns[:, 2], patterns[:, 4]], (nb_edges_graph_a, nb_edges_graph_b)
        )
        e_a_j_ab_jb = np.ravel_multi_index(
            [patterns[:, 2], patterns[:, 5]], (nb_edges_graph_a, nb_edges_graph_b)
        )

        theta_i_a_ib_ab = jax.ops.segment_sum(
            np.take(k.ntk, e_i_a_ib_ab), e_i_j_ib_jb, num_segments
        )
        theta_i_a_ab_jb = jax.ops.segment_sum(
            np.take(k.ntk, e_i_a_ab_jb), e_i_j_ib_jb, num_segments
        )
        theta_a_j_ib_ab = jax.ops.segment_sum(
            np.take(k.ntk, e_a_j_ib_ab), e_i_j_ib_jb, num_segments
        )
        theta_a_j_ab_jb = jax.ops.segment_sum(
            np.take(k.ntk, e_a_j_ab_jb), e_i_j_ib_jb, num_segments
        )

        thetas_linear = np.zeros((theta_i_a_ab_jb.shape[0], 1))
        thetas_linear = np.append(thetas_linear, np.expand_dims(theta_i_a_ib_ab, 1), 1)
        thetas_linear = np.append(thetas_linear, np.expand_dims(theta_i_a_ab_jb, 1), 1)
        thetas_linear = np.append(thetas_linear, np.expand_dims(theta_a_j_ib_ab, 1), 1)
        thetas_linear = np.append(thetas_linear, np.expand_dims(theta_a_j_ab_jb, 1), 1)

        theta_linear = np.sum(thetas_linear, 1)
        theta = np.reshape(theta_linear, k.ntk.shape)
        thera = theta + k.ntk
        
        return k.replace(ntk=theta, is_gaussian=True, is_input=False)

In [17]:
# define a toy 2x graph
# 0---2 

edge_list_2 = np.array([[0,0],
                      [1,1],
                      [0,1],
                      [1,0]])

A = to_dense(np.transpose(edge_list_2), 2)
As = np.expand_dims(A, 0)
As_int = neigbourhood_intersections(As)
As_pattern_2 = np.transpose(np.array(np.nonzero(As_int)))


print(As_pattern_2.shape)

# define a toy kernel matrix
# (1, 1, 1, 1, 9, 9)
toy_kernel_2 = np.array(range(4*4))
toy_kernel_2 = np.reshape(toy_kernel_2, (4,4))

toy_kernel_2 = np.expand_dims(toy_kernel_2, 0)
toy_kernel_2 = np.expand_dims(toy_kernel_2, 0)
toy_kernel_2 = np.expand_dims(toy_kernel_2, 0)
toy_kernel_2 = np.expand_dims(toy_kernel_2, 0)

toy_kernel_2.shape

(8, 4)


(1, 1, 1, 1, 4, 4)

In [16]:
# define a toy 2x graph
# 0---2 

toy_kernel_2_new = np.array(range(3*3))
toy_kernel_2_new = np.reshape(toy_kernel_2_new, (3,3))

toy_kernel_2_new = np.expand_dims(toy_kernel_2_new, 0)
toy_kernel_2_new = np.expand_dims(toy_kernel_2_new, 0)
toy_kernel_2_new = np.expand_dims(toy_kernel_2_new, 0)
toy_kernel_2_new = np.expand_dims(toy_kernel_2_new, 0)

toy_kernel_2_new

Array([[[[[[0, 1, 2],
           [3, 4, 5],
           [6, 7, 8]]]]]], dtype=int32)

In [18]:
edge_list_2 = np.array([[0,0],
                      [1,1],
                      [0,1],
                      [1,0]])

pattern, edge_list = twl_sparse_pattern(edge_list_2, 2)

pattern

Array([[0, 0, 0],
       [0, 2, 2],
       [1, 1, 1],
       [1, 2, 2],
       [2, 0, 2],
       [2, 2, 1]], dtype=int32)

The output of the kernel function for the toy kernel ($\Theta_{(i,j,i',j')}^{(l}$) must be:
$$
    \Theta_{(i,j,i',j')}^{(l)}
    =
    \sum_{v_a \in \Gamma_G^r(v_i) \cap \Gamma_G^r(v_j)}
    \sum_{v_a' \in \Gamma_G^r(v_i') \cap \Gamma_G^r(v_j')}
    \Theta_{(i,a,i',a')}^{(l-1)}
    +
    \Theta_{(a,j,a',j')}^{(l-1)}
    +
    \Theta_{(i,a,a',j')}^{(l-1)}
    +
    \Theta_{(a,j,i',a')}^{(l-1)}
$$
We can calculate this manually with the following table:

In [255]:
np.reshape(np.squeeze(toy_kernel_2), (2,2,2,2))

Array([[[[ 0,  1],
         [ 2,  3]],

        [[ 4,  5],
         [ 6,  7]]],


       [[[ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15]]]], dtype=int32)

In [23]:
k = nt.Kernel(nngp=None, 
          ntk=toy_kernel_2_new,
          cov1=None,
          cov2=None, 
          x1_is_x2=None, 
          is_gaussian=None, 
          is_reversed=None, 
          is_input=None, 
          diagonal_batch=None, 
          diagonal_spatial=None, 
          shape1=None, 
          shape2=None, 
          batch_axis=None, 
          channel_axis=None, 
          mask1=None, 
          mask2=None)

kernel_matrix = kernel_fn_new(k=k, pattern=(pattern, pattern))

3
3


In [24]:
np.squeeze(kernel_matrix.ntk)


Array([[64., 72., 68.],
       [88., 96., 92.],
       [76., 84., 80.]], dtype=float32)